In [1]:
#import all the libraries we need
from bs4 import BeautifulSoup
import requests
import MySQLdb as mdb
import datetime
from datetime import date, datetime, timedelta
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import datetime
import time
import arrow
import re
import requests
import json
from slackclient import SlackClient
from tabulate import tabulate

# Make the graphs a bit prettier, and bigger
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])
plt.rcParams['figure.figsize'] = (15, 5)

In [2]:
#!sudo pip3 install -U tabulate

In [3]:
def plot_recent():

    conn_string_realmdata = 'mysql://{user}:{password}@{host}:{port}/{db}'.format(
        user='root', 
        password='dwdstudent2015', 
        host = 'localhost', 
        port=3306, 
        db='realmdata'
    )
    engine_realmdata = create_engine(conn_string_realmdata)

    query2 = '''
    SELECT char_stats , count(*) 
    FROM recent_deaths 
    WHERE char_stats != "hidden"
    GROUP BY `char_stats`
    '''
    df_realmdata = pd.read_sql(query2, con=engine_realmdata)
    df_realmdata2 = df_realmdata.set_index('char_stats')
    orange = df_realmdata2.plot(kind="bar")
    #this query is stored into a panda dataframe for graphing

    # Save the plot and return its url
    fig = orange.get_figure()
    filename = 'plots/dstrib.png'
    fig.savefig(filename)
    plt.close(fig)
    
    url = 'http://52.44.93.5:5000/' + filename
    return url

plot_recent()

'http://52.44.93.5:5000/plots/dstrib.png'

In [4]:
def plot_recent_armor():
    
    conn_string_realmdata = 'mysql://{user}:{password}@{host}:{port}/{db}'.format(
        user='root', 
        password='dwdstudent2015', 
        host = 'localhost', 
        port=3306, 
        db='realmdata'
    )
    engine_realmdata = create_engine(conn_string_realmdata)
    
    query3 = '''
    SELECT armor , count(*)
    FROM recent_deaths
    WHERE armor != "hidden"
    GROUP BY armor
    HAVING count(*) > 5
    ORDER BY count(*) DESC
    '''
    armor = pd.read_sql(query3, con = engine_realmdata)
    armor2 = armor.set_index("armor")
    armfig = armor2.plot(kind = "bar")
    
    fig = armfig.get_figure()
    filename = 'plots/armor.png'
    fig.savefig(filename)
    plt.close(fig)
    
    url = 'http://52.44.93.5:5000/' + filename
    return url

plot_recent_armor()

'http://52.44.93.5:5000/plots/armor.png'

In [5]:
def plot_recent_weapon():
    
    conn_string_realmdata = 'mysql://{user}:{password}@{host}:{port}/{db}'.format(
        user='root', 
        password='dwdstudent2015', 
        host = 'localhost', 
        port=3306, 
        db='realmdata'
    )
    engine_realmdata = create_engine(conn_string_realmdata)
    
    query3 = '''
    SELECT weapon , count(*)
    FROM recent_deaths
    WHERE weapon != "hidden"
    GROUP BY weapon
    HAVING count(*) > 3
    ORDER BY count(*) DESC
    '''
    weapondata = pd.read_sql(query3, con = engine_realmdata)
    weapondata2 = weapondata.set_index("weapon")
    orange = weapondata2.plot(kind = "bar")

    fig = orange.get_figure()
    filename = 'plots/weapons.png'
    fig.savefig(filename)
    plt.close(fig)
    
    url = 'http://52.44.93.5:5000/' + filename
    return url

plot_recent_weapon()

'http://52.44.93.5:5000/plots/weapons.png'

In [6]:
def plot_recent_fame():
    urllist = []
    conn_string_realmdata = 'mysql://{user}:{password}@{host}:{port}/{db}'.format(
        user='root', 
        password='dwdstudent2015', 
        host = 'localhost', 
        port=3306, 
        db='realmdata'
    )
    engine_realmdata = create_engine(conn_string_realmdata)
    
    query5 = '''
    SELECT ((total_fame - base_fame)/total_fame) as precent_bonus
    FROM recent_deaths
    WHERE base_fame > 0 
    LIMIT 100
    '''
    query6 = '''
    SELECT base_fame, total_fame
    FROM recent_deaths
    WHERE base_fame > 0 
    LIMIT 100
    '''
    fame = pd.read_sql(query5, con = engine_realmdata)
    apple = fame.plot()
    applefig = apple.get_figure()
    filename1 = 'plots/famepercent.png'
    applefig.savefig(filename1)
    plt.close(applefig)
    url1 = 'http://52.44.93.5:5000/' + filename1
    
    fame2 = pd.read_sql(query6, con = engine_realmdata)
    orange = fame2.plot()
    orangefig = orange.get_figure()
    filename2 = 'plots/fame.png'
    orangefig.savefig(filename2)
    plt.close(orangefig)
    url2 = 'http://52.44.93.5:5000/' + filename2
    
    urllist.append(url1)
    urllist.append(url2)
    
    return urllist

plot_recent_fame()

['http://52.44.93.5:5000/plots/famepercent.png',
 'http://52.44.93.5:5000/plots/fame.png']

In [7]:
#this code creates the messages that the slackbot will return
def create_message(username, info):
    attachments = []
    #if the user doesn't ask for the above an error is given
    if info == "options":
        message = "Start every command with 'give me _______' \n"
        message += "fill in the blank with: \n"
        message += "'recent deaths'  :  for the last 100 recent deaths \n"
        message += "'recent deaths graph'  :  for a distribution of character stats \n"
        message += "'recent armor graph'  :  for a graph of character armor \n"
        message += "'recent weapons graph'  :  for a graph of characer weapons \n"
        message += "'recent fame graph'  :  for information regarding fame gain \n"
    elif info == "recent deaths":
        message = "Visit this link for the last 100 recent deaths:\n"
        message += "http://52.44.93.5:5000/realm"
    elif info == "recent deaths graph":
        url = plot_recent()
        attachment = {
            "image_url": url,
            "title": str(datetime.datetime.now())
        }
        attachments.append(attachment)
        message = "Stat distribution of last 1000 deaths"
    elif info == "recent armor graph":
        url = plot_recent_armor()
        attachment = {
            "image_url": url,
            "title": str(datetime.datetime.now())
        }
        attachments.append(attachment)
        message = "Most common armors used by recent characters"
    elif info == "recent weapons graph":
        url = plot_recent_weapon()
        attachment = {
            "image_url": url,
            "title": str(datetime.datetime.now())
        }
        attachments.append(attachment)
        message = "Most common weapons used by recent characters"
    elif info == "recent fame graph":
        url = plot_recent_fame()
        attachment1 = {
            "image_url": url[0],
            "title": str(datetime.datetime.now())
        }
        attachment2 = {
            "image_url": url[1],
            "title": str(datetime.datetime.now())
        }
        attachments.append(attachment1)
        attachments.append(attachment2)
        message = "Fame gain/bonuses of past 100 characters"
    else:
        message =  "Thank you @{u} for asking! \n".format(u=username)
        message += "Unfortunately I did not understand what you are asking for.\n"
        message += "Try using 'give me options', to see what you can do :)"
    return message,attachments #this function uses a regex to find which word


In [8]:
#this function uses a regex to find which word
def extract_swag(message_text):
    regex_expression = 'give me (.+)'
    regex= re.compile(regex_expression)
    matches = regex.finditer(message_text)
    for match in matches:
        return match.group(1)
    # if there were no matches, return None
    return None


In [9]:
#this function checks for users prompting the bot
def message_matches(user_id, message_text):
    regex_expression = '.*@' + user_id + '.*bot.*'
    regex = re.compile(regex_expression)
    # Check if the message text matches the regex above
    match = regex.match(message_text)
    # returns true if the match is not None (ie the regex had a match)
    return match != None 

In [ ]:
# Read the access token from the file and create the Slack Client

secrets_file = 'slack_secret.json'
f = open(secrets_file, 'r') 
content = f.read()
f.close()

auth_info = json.loads(content)
auth_token = auth_info["access_token"]
bot_user_id = auth_info["user_id"]

from slackclient import SlackClient
sc = SlackClient(auth_token)


In [ ]:
# this code runs the slackbot

if sc.rtm_connect():
    # We are going to be polling the Slack API for recent events continuously
    while True:
        # We are going to wait 1 second between monitoring attempts
        time.sleep(1)
        # If there are any new events, we will get a response. If there are no events, the response will be empty
        response = sc.rtm_read()
        for item in response:
            # Check that the event is a message. If not, ignore and proceed to the next event.
            if item.get("type") != 'message':
                continue
                
            # Check that the message comes from a user. If not, ignore and proceed to the next event.
            if item.get("user") == None:
                continue
            
            # Check that the message is asking the bot to do something. If not, ignore and proceed to the next event.
            message_text = item.get('text')
            if not message_matches(bot_user_id, message_text):
                continue
                
            # Get the username of the user who asked the question
            response = sc.api_call("users.info", user=item["user"])
            username = response['user'].get('name')
            
            # Extract the station name from the user's message
            swag = extract_swag(message_text)

            # Prepare the message that we will send back to the user
            message, attachments = create_message(username, swag)

            # Post a response to the #bots channel
            sc.api_call("chat.postMessage", channel="#bots", text=message, attachments = attachments)